In [1]:
# load the overlay 
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
import numpy as np
import requests
import sys
import time

base = BaseOverlay("base.bit")
Mode = VideoMode(1280,720,24) 

#initialize camera from openCV 
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

if(not videoIn.isOpened()): 
    print("Error: Camera is not open")
    sys.exit()

In [2]:
face_cascade = cv2.CascadeClassifier('/home/xilinx/jupyter_notebooks/base/video/data/haarcascade_frontalface_default.xml')
body_cascade = cv2.CascadeClassifier('/home/xilinx/jupyter_notebooks/base/video/data/haarcascade_upperbody.xml')
 
if (body_cascade.empty()):
    print("Failed to load cascade from file.")

try:
    while True:   
        # read next image
        ret, frame_vga = videoIn.read()
        outframe = hdmi_out.newframe()
        outframe[:] = frame_vga
        gray_image = cv2.cvtColor(outframe, cv2.COLOR_BGR2GRAY)
        # detect multiscale returns x, y, w, h
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.4, minNeighbors=5, minSize=(75, 75))
        upper_bodies = body_cascade.detectMultiScale(gray_image, 1.08, 1, minSize=(200, 200))

        if (ret):    
            if(len(faces)>0 or len(upper_bodies)>0):
                frameList = outframe.tolist()
                url = 'http://192.168.1.10:9595/uploadFrame/'
                data = {'param': '911396a7-de99-49e0-b23d-643f48f08348', 'arr': frameList}
                response = requests.post(url, json=data)
                time.sleep(3)
        else:
            # video input disconnected
            videoIn.release()
            hdmi_out.stop()
            del hdmi_out
            sys.exit()

except KeyboardInterrupt:
    videoIn.release()
    hdmi_out.stop()
    del hdmi_out
